Create evaluations of the likelihood for plotting purposes.
Evaluate likelihood on a grid of prm, ic combinations.

In [1]:
using Pkg
Pkg.activate("C:/Users/lisah/Documents/Repos/ma-code")

  Activating project at `C:\Users\lisah\Documents\Repos\ma-code`


In [2]:
using DifferentialEquations, CSV, DataFrames, Plots, Distributions

In [3]:
abstract type AbstractHyperprm end

struct Hyperprm <: AbstractHyperprm
    w0::Float64
    n0::Float64
    a::Float64
    m::Float64
    M::Int64
    noise::Float64
end

In [4]:
# define and solve model

function klausmeier!(du,u,p,t)
    du[1] = -u[1] - u[1] * u[2]^2 + p[1] # water compartment
    du[2] = u[1] * u[2]^2 - p[2] * u[2] # biomass compartment
end

function sol_klausmeier(hprm::Hyperprm)
    u0 = [hprm.w0; hprm.n0]
    p = [hprm.a; hprm.m]
    tspan = (0.0, hprm.M-1)

    prob = ODEProblem(klausmeier!, u0, tspan, p)
    sol = solve(prob,
        saveat=1.0  # consider specific time points
    )

    return DataFrame(time=sol.t, w=sol[1, :], n=sol[2, :])
end

sol_klausmeier (generic function with 1 method)

In [5]:
# we evaluate likelihood for a in (0,2), n0 in (0,4) on an uniformly spaced grid

function create_grid()
    a_vals = 0.0:0.1:2.0
    n_vals = 0.0:0.1:4.0

    grid = [(a, n) for n in n_vals, a in a_vals] 
    return grid
end

create_grid (generic function with 1 method)

In [6]:
# add Gaussian noise to data

function randomize_data(df::DataFrame, noise::Float64)
    if noise == 0.0
        return df
    else
        df[!, "w"] .= df[!, "w"] .+ rand(Normal(0, noise), nrow(df))
        df[!, "n"] .= df[!, "n"] .+ rand(Normal(0, noise), nrow(df))
        return df
    end
end

randomize_data (generic function with 1 method)

In [7]:
function compute_ll(hprm::Hyperprm, true_val::DataFrame)
    pred_val = sol_klausmeier(hprm)
    ll = -0.5 * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2) - 0.5 * sum((true_val[:,"w"] - pred_val[:,"w"]) .^2) # add up ll for both trajectories
    #ll = -0.5 * sum((true_val[:,"n"] - pred_val[:,"n"]) .^2)
    return ll
end

compute_ll (generic function with 1 method)

In [8]:
function store_ll_data(w0::Float64,n0::Float64,a::Float64,m::Float64,M::Int64,noise::Float64,df::DataFrame)
    CSV.write("c:/Users/lisah/Documents/Repos/ma-code/data/likelihood/m0.45/ll_$(w0)_$(n0)_$(a)_$(m)_$(M)_$(noise).csv", df)
end

store_ll_data (generic function with 1 method)

In [9]:
# generates and stores data for one prm combination. Run this for all a,n0,M,noise combinations wanted

function gen_ll_evals_for_hprm_comb(hprm_true::Hyperprm)

    grid = create_grid()
    sol_true = sol_klausmeier(hprm_true) # returns df
    sol_true = randomize_data(sol_true, hprm_true.noise) # include noise

    ll = zeros(41, 21)

    for i in range(1, 41)
        for j in range(1, 21)
            #eval model for each point on grid
            pt = grid[i,j]
            hprm = Hyperprm(hprm_true.w0, pt[2], pt[1], hprm_true.m, hprm_true.M, hprm_true.noise) #w0,n0,a,m,M
            #eval likelihood
            ll_val = compute_ll(hprm, sol_true)
            
            ll[i,j] = ll_val
        end
    end
    
    #return data frame
    a_eval_pts = string.(0.0:0.1:2.0)
    df_ll = DataFrame(ll, a_eval_pts)

    return df_ll
end

gen_ll_evals_for_hprm_comb (generic function with 1 method)

In [10]:
# generate and store data for all hyperparameter combinations

function gen_all_ll_data(index_combos, M_vals, noise_vals)
    for ind in index_combos
        for M in M_vals
            for noise in noise_vals
                a_ind = ind[1]
                n0_ind = ind[2]
                hprm = Hyperprm(w0, n0_vals[n0_ind], a_vals[a_ind], m, M, noise)
                df_ll = gen_ll_evals_for_hprm_comb(hprm)
                store_ll_data(w0, n0_vals[n0_ind], a_vals[a_ind], m, M, noise, df_ll)
            end
        end
    end
end

gen_all_ll_data (generic function with 1 method)

Prm combinations are chosen similarly to the reference paper and result in the following values. For now consider 5 index combinations, namely [1,1], [2,4], [3,5], [4,2], [4,3]

In [11]:
a_vals = [0.1, 0.9, 1.1, 1.7] 
n0_vals = [0.4, 0.5, 1.0, 1.3, 2.3]

index_combos = [[1,1], [2,4], [3,5], [4,2], [4,3]]

5-element Vector{Vector{Int64}}:
 [1, 1]
 [2, 4]
 [3, 5]
 [4, 2]
 [4, 3]

In [12]:
# generate data for single case

# ind = [2,4]
# M = 1000
# noise = 0.0
# m = 0.45 # fix for now as given by literature
# w0 = 0.9 # Q: which value?

# a_ind = ind[1]
# n0_ind = ind[2]
# hprm = Hyperprm(w0, n0_vals[n0_ind], a_vals[a_ind], m, M, noise)
# df_ll = gen_ll_evals_for_hprm_comb(hprm)
 
# store_ll_data(w0, n0_vals[n0_ind], a_vals[a_ind], m, M, noise, df_ll)

In [13]:
# generate all data 

M_vals = [10,100,500,1000]
noise_vals = [0.0,0.1,1.0]
m = 0.45 # fix for now as given by literature
w0 = 0.95 # mean(a_vals)

gen_all_ll_data(index_combos, M_vals, noise_vals)